<h3>Import necessary packages </h3>


In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

<h3> Read pre-processsed file </h3>


In [27]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('Merged_df.csv', index_col=0) 

/var/folders/pc/cw9rfzrs40s854nlyd3h7q_h0000gn/T/ipykernel_99924/1463368140.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Merged_df.csv', index_col=0)


In [28]:
df

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode,Movie_Title,Movie_Year,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067,One Flew Over the Cuckoo's Nest,1975,510.0,One Flew Over the Cuckoo's Nest,8.420,9650.0,Released,1975-11-19,108981275.0,133.0,False,/qUq3QTr2KLvGIcN0GaaaYx9bbyH.jpg,3000000.0,NaN,tt0073486,en,One Flew Over the Cuckoo's Nest,A petty criminal fakes insanity to serve his s...,36.673,/3jcbDmRFiQ83drXNOvRDeKHxS0C.jpg,"If he's crazy, what does that make you?",Drama,Fantasy Films,United States of America,English,1975.0,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067,James and the Giant Peach,1996,10539.0,James and the Giant Peach,6.556,1196.0,Released,1996-04-12,28946127.0,79.0,False,/6kTLm7zfwKAgAcGnaVU0uT4QYug.jpg,38000000.0,http://movies.disney.com/james-and-the-giant-p...,tt0116683,en,James and the Giant Peach,When the young orphan boy James spills a magic...,27.497,/nl2oB6EbD1fHFuP2TLUHDtqs7Ux.jpg,Adventures this big don't grow on trees.,"Adventure, Animation, Family, Fantasy","Allied Filmmakers, Walt Disney Pictures",United States of America,English,1996.0,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067,My Fair Lady,1964,11113.0,My Fair Lady,7.500,1161.0,Released,1964-10-21,72661442.0,170.0,False,/xMoQjqa3JUc2unO4nF9PmM1JTQu.jpg,17000000.0,NaN,tt0058385,en,My Fair Lady,A snobbish phonetics professor agrees to a wag...,19.547,/bTXVc29lGSNclf94VIZ49W4gGKl.jpg,The loverliest motion picture of them all!,"Romance, Drama, Comedy",Warner Bros. Pictures,United States of America,English,1964.0,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067,Erin Brockovich,2000,462.0,Erin Brockovich,7.417,2855.0,Released,2000-03-17,256271286.0,131.0,False,/3HmLKhahTxX2ALTWlagr92pZrU7.jpg,52000000.0,NaN,tt0195685,en,Erin Brockovich,A twice-divorced mother of three who sees an i...,22.495,/xbMEHLu36LyptPD29FxVTndpYZ1.jpg,She brought a small town to its feet and a hug...,Drama,Jersey Films,United States of America,English,2000.0,Erin Brockovich (2000)
4,1,2355,5,978824291,A Bug's Life (1998),Animation|Children's|Comedy,F,1,10,48067,A Bug's Life,1998,9487.0,A Bug's Life,6.969,8541.0,Released,1998-11-25,363258859.0,95.0,False,/hwwFyowfcbLRVmRBOkvnABBNIOs.jpg,120000000.0,http://movies.disney.com/a-bugs-life,tt0120623,en,A Bug's Life,"On behalf of ""oppressed bugs everywhere,"" an i...",58.168,/Ah3J9OJVc2CNCuH2zMydXy9fmIC.jpg,An epic presentation of miniature proportions.,"Adventure, Animation, Comedy, Family","Pixar, Walt Disney Pictures",United States of America,English,1998.0,A Bug's Life (1998)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994980,6040,1094,5,956704887,The Crying Game (1992),Drama|Romance|War,M,25,6,11106,The Crying Game,1992,11386.0,The Crying Game,6.860,666.0,Released,1992-09-02,62548947.0,112.0,False,/htBKlZaVLn7hnz36AdvVR53LwB8.jpg,3705538.0,http://www.miramax.com/movie/the-crying-game/,tt0104036,en,The Crying Game,Irish Republican Army member Fergus forms an u...,19.329,/ynEfiiwS0dosbPjzGqBLo7FXMlD.jpg,Play At Your Own Risk.,"Crime, Drama, Thriller","Palace Pictures, Nippon Film Development and F...","Ireland, Japan, United Kingdom",English,1992.0,The Crying Game (1992)
994981,6040,1094,5,956704887,The Crying Game (1992),Drama|Romance|War,M,25,6,11106,The Crying Game,1992,1214190.0,The Crying Game,0.000,0.0,Released,1992-05-05,0.0,34.0,False,NaN,0.0,NaN,tt0544892,en,The Crying Game,Roy Brush has aspirations to be a great f

<h3> Normalize age </h3>

In [29]:
scaler = StandardScaler()
df['Normalized_Age'] = scaler.fit_transform(df[['Age']])

<h3>One hot encoding</h3>

In [30]:
encoder = OneHotEncoder()
user_attrs = df[['UserID', 'Gender', 'Occupation']].drop_duplicates().reset_index(drop=True)
encoded_attrs = encoder.fit_transform(user_attrs[['Gender', 'Occupation']])
encoded_attrs_df = pd.DataFrame(encoded_attrs.toarray(), index=user_attrs.index)
encoded_attrs_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6036,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3> Combining the original UserID column with the one-hot encoded attributes of Gender and Occupation into a single DataFrame. </h3>

In [31]:
user_attrs_final = pd.concat([user_attrs[['UserID']], encoded_attrs_df], axis=1)
user_attrs_final

,UserID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6036,6037,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,6038,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,6039,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3> Creating UserID and MovieID mappings to ensure data consistency and compatability </h3>

In [32]:
user_ids = df['UserID'].unique()
user_id_map = {i: idx for idx, i in enumerate(user_ids)}
movie_ids = df['MovieID'].unique()
movie_id_map = {i: idx for idx, i in enumerate(movie_ids)}

df['UserID'] = df['UserID'].map(user_id_map)
df['MovieID'] = df['MovieID'].map(movie_id_map)
user_attrs_final['UserID'] = user_attrs_final['UserID'].map(user_id_map)



In [46]:
user_attrs_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
UserID,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6036,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3> Split data into training and testing </h3>

In [33]:
train, test = train_test_split(df, test_size=0.2, random_state=42)


<h3> Collaborative filtering using deep learning (Neural Collaborative Filtering model) </h3>

In [35]:
def RecommenderNet(num_users, num_movies, num_user_attrs, embedding_size=50):
    # User and Movie Embedding
    user_input = Input(shape=(1,), name='user_input')
    movie_input = Input(shape=(1,), name='movie_input')
    
    # Updated Embedding layers without 'input_length'
    user_emb = Embedding(output_dim=embedding_size, input_dim=num_users, name='user_embedding')(user_input)
    movie_emb = Embedding(output_dim=embedding_size, input_dim=num_movies, name='movie_embedding')(movie_input)
    
    # Flatten embeddings
    user_vec = Flatten()(user_emb)
    movie_vec = Flatten()(movie_emb)
    
    # Dot product of embeddings for collaborative filtering prediction
    cf_pred = Dot(axes=1)([user_vec, movie_vec])
    
    # User attributes input
    attrs_input = Input(shape=(num_user_attrs,), name='attrs_input')
    attrs_dense = Dense(128, activation='relu')(attrs_input)
    
    # Concatenate CF prediction with user attributes
    concat = Concatenate()([cf_pred, attrs_dense])
    x = Dense(128, activation='relu')(concat)
    x = Dense(1)(x)
    
    model = Model(inputs=[user_input, movie_input, attrs_input], outputs=x)
    model.compile(optimizer=Adam(0.001), loss='mse')
    
    return model


<h3> Initializing deep learning model </h3>

In [36]:
num_users = len(user_ids)
num_movies = len(movie_ids)
num_user_attrs = encoded_attrs_df.shape[1]

model = RecommenderNet(num_users, num_movies, num_user_attrs)



,UserID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6035,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6036,6036,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,6037,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,6038,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
train

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode,Movie_Title,Movie_Year,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year,Normalized_Age
477995,2945,1209,3,975828421,Things to Do in Denver when You're Dead (1995),Crime|Drama|Romance,M,35,16,92630,Things to Do in Denver when You're Dead,1995,400.0,Things to Do in Denver When You're Dead,6.496,338.0,Released,1995-12-01,529766.0,116.0,False,/7fEOKdQIfbkOxy0enLtpyPGUsTm.jpg,8000000.0,NaN,tt0114660,en,Things to Do in Denver When You're Dead,Five different criminals face imminent death a...,19.110,/oPp6Gbrasox66WyMvPS0k8OakQf.jpg,Protect. Love. Honor. Avenge.,"Drama, Crime",Miramax,United States of America,"French, English",1995.0,Things to Do in Denver When You're Dead (1995),0.446453
114076,745,148,2,975467135,Gone with the Wind (1939),Drama|Romance|War,F,18,19,95019,Gone with the Wind,1939,770.0,Gone with the Wind,7.965,3691.0,Released,1939-12-15,402352579.0,238.0,False,/ft8WRF2xqEwwGWa59naDUybKTAx.jpg,4000000.0,NaN,tt0031381,en,Gone with the Wind,The spoiled daughter of a Georgia plantation o...,46.849,/lNz2Ow0wGCAvzckW7EOjE03KcYv.jpg,The greatest romance of all time!,"Drama, War, Romance","Selznick International Pictures, Metro-Goldwyn...",United States of America,English,1939.0,Gone with the Wind (1939),-0.999621
272217,1648,795,2,974712016,The Nutty Professor (1996),Comedy|Fantasy|Romance|Sci-Fi,M,18,19,98034,The Nutty Professor,1996,9327.0,The Nutty Professor,5.625,2697.0,Released,1996-06-28,128769345.0,95.0,False,/gHFuLHxruielBcMEZx1WowxbRWS.jpg,54000000.0,NaN,tt0117218,en,The Nutty Professor,"Eddie Murphy stars as shy Dr. Sherman Klump, a...",16.421,/fMtb5aZoLRNbMnCkatFsTmPRfl5.jpg,"Inside Sherman Klump, a party animal is about ...","Fantasy, Comedy, Romance, Science Fiction","Universal Pictures, Imagine Entertainment",United States of America,English,1996.0,The Nutty Professor (1996),-0.999621
875351,5316,283,5,960905686,Pi (1998),Sci-Fi|Thriller,F,25,0,02138,Pi,1998,473.0,Pi,7.111,2021.0,Released,1998-07-10,3221152.0,84.0,False,/rYpGznw5UExo3ojqRK8eRIfqoEJ.jpg,60000.0,NaN,tt0138704,en,Pi,A mathematical genius discovers a link between...,18.313,/dET9ULtaMJEUnfylQE1n5LC9gr7.jpg,"There will be no order, only chaos","Mystery, Drama, Thriller","Harvest Filmworks, Plantain Films, Protozoa Pi...",United States of America,English,1998.0,Pi (1998),-0.404179
442204,2741,40,4,973220893,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,M,25,17,36092,Star Wars: Episode IV - A New Hope,1977,11.0,Star Wars,8.204,19155.0,Released,1977-05-25,775398007.0,121.0,False,/zqkmTXzjkAgXmEWLRsY4UpTWCeo.jpg,11000000.0,http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,88.559,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,"A long time ago in a galaxy far, far away...","Adventure, Action, Science Fiction","Lucasfilm Ltd., 20th Century Fox",United States of America,English,1977.0,Star Wars (1977),-0.404179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259005,1591,1745,5,974737853,The Apartment (1960),Comedy|Drama,M,35,7,50111,The Apartment,1960,284.0,The Apartment,8.222,2016.0,Released,1960-06-21,25000000.0,125.0,False,/vRTKNKNWLZ22fAgPa5kY8wT2b1F.jpg,3000000.0,NaN,tt0053604,en,The Apartment,Bud Baxter is a minor clerk in a huge New York...,24.186,/hhSRt1KKfRT0yEhEtRW3qp31JFU.jpg,"Movie-wise, there has never been anything like...","Comedy, Drama, Romance","The Mirisch Company, United Artists",United States of America,English,1960.0,The Apartment (1960),0.446453
365599,2144,99,5,974627359,"Misérables, Les (1998)",Drama,M,18,10,32750,"Misérables, Les",1998,4415.0,Les Miséra

<h3> Preparing the training data </h3>

In [38]:
user_attrs_final.set_index('UserID', inplace=True)
user_input_train = train['UserID'].values
movie_input_train = train['MovieID'].values
attrs_input_train = user_attrs_final.loc[user_input_train].values
rating_train = train['Rating'].values


<h3> Training the model, target output is ratings  </h3>

In [39]:
history = model.fit([user_input_train, movie_input_train, attrs_input_train], rating_train,
                    batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
9957/9957 ━━━━━━━━━━━━━━━━━━━━ 31s 3ms/step - loss: 1.2485 - val_loss: 0.8222
Epoch 2/5
9957/9957 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - loss: 0.6538 - val_loss: 0.8304
Epoch 3/5
9957/9957 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - loss: 0.4941 - val_loss: 0.8573
Epoch 4/5
9957/9957 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - loss: 0.4149 - val_loss: 0.8901
Epoch 5/5
9957/9957 ━━━━━━━━━━━━━━━━━━━━ 31s 3ms/step - loss: 0.3724 - val_loss: 0.9216


<h3> Finding loss on testing dataset </h3>

In [40]:
user_input_test = test['UserID'].values
movie_input_test = test['MovieID'].values
attrs_input_test = user_attrs_final.loc[user_input_test].values  
rating_test = test['Rating'].values
evaluation = model.evaluate([user_input_test, movie_input_test, attrs_input_test], rating_test)


6223/6223 ━━━━━━━━━━━━━━━━━━━━ 2s 315us/step - loss: 0.9211


<h3> The aim of the functions below is to output a list of recommended movies for a particular user input </h3>

In [42]:
def get_unseen_movies(user_id, all_movie_ids, ratings_df):

    seen_movies = ratings_df[ratings_df['UserID'] == user_id]['MovieID'].unique()

    unseen_movies = [movie for movie in all_movie_ids if movie not in seen_movies]
    return unseen_movies

def predict_ratings_for_user_with_titles(model, user_id, unseen_movies, user_attrs, movie_titles_df, num_recommendations=5):
    user_input = np.array([user_id] * len(unseen_movies))
    movie_input = np.array(unseen_movies)
    attrs_input = np.array([user_attrs] * len(unseen_movies))
    predicted_ratings = model.predict([user_input, movie_input, attrs_input]).flatten()

    top_indices = predicted_ratings.argsort()[-num_recommendations:][::-1]  
    top_movie_ids = movie_input[top_indices]
    top_predicted_ratings = predicted_ratings[top_indices]
    recommendations_df = pd.DataFrame({
        'MovieID': top_movie_ids,
        'PredictedRating': top_predicted_ratings
    })
    
    recommendations_with_titles = recommendations_df.merge(movie_titles_df, on='MovieID')
    
    return recommendations_with_titles[['MovieID', 'Title', 'PredictedRating']]







In [43]:
all_movie_ids = df['MovieID'].unique()
ratings_df = df[['UserID', 'MovieID', 'Rating']].copy()
movie_titles_df = df[['MovieID', 'Title']].drop_duplicates().set_index('MovieID')

In [44]:
def final_prediction(user_id, model = model, all_movie_ids = all_movie_ids, ratings_df = ratings_df, movie_titles_df = movie_titles_df):
    unseen_movies = get_unseen_movies(user_id, all_movie_ids, ratings_df)
    user_attrs = encoded_attrs_df.loc[user_id].values
    recommendations_with_titles = predict_ratings_for_user_with_titles(model, user_id, unseen_movies, user_attrs, movie_titles_df, num_recommendations=5)
    return recommendations_with_titles



<h3> Finally, export a top 1 recommended movie for each user to a csv file for further use in our content filtering models </h3>

In [25]:
results = []
for i in range(0, 6040):
    result = final_prediction(i).iloc[0]  
    results.append(result)

results_final = pd.DataFrame(results)

results_final.to_csv('final_predictions.csv', index=False)



114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 529us/step
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step
111/111 ━━━